## Algorithmes variationnels

Ce cours couvre les spécificités des algorithmes variationnels et des algorithmes quantiques-classiques hybrides à court terme basés sur le théorème variationnel de la mécanique quantique. Ces algorithmes peuvent tirer parti de l'utilité fournie par les ordinateurs quantiques non tolérants aux pannes d'aujourd'hui, ce qui en fait des candidats idéaux pour obtenir [un avantage quantique](gloss:advantage) .

Tout au long de ce cours, nous explorerons :

- Chaque étape du workflow de conception d'algorithmes variationnels
- Les compromis associés à chaque étape
- Comment utiliser [les primitives Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html) pour optimiser la vitesse et la précision

Bien que ce cours soit destiné à être un point de départ pour les chercheurs et les développeurs pour explorer l'utilité des ordinateurs quantiques, n'hésitez pas à explorer les connaissances théoriques et fondamentales sur l'informatique quantique en général dans les [bases de l'information et du calcul quantiques](https://qiskit.org/learn/course/basics-quantum-information/) (également disponible [en série de vidéos YouTube](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqKKMXX4vbi4EB1uaErFMSO) ).

## Workflow hybride simplifié

![Flux variationnel](images/variational_workflow.png)

Les algorithmes variationnels comprennent plusieurs composants modulaires qui peuvent être combinés et optimisés en fonction des progrès de l'algorithme, du logiciel et du matériel. Cela inclut une *fonction de coût* qui décrit un problème spécifique avec un ensemble de paramètres, un *ansatz* pour exprimer l'espace de recherche avec ces paramètres et un *optimiseur* pour explorer de manière itérative l'espace de recherche. Lors de chaque itération, l'optimiseur évalue la fonction de coût avec les paramètres actuels et sélectionne les paramètres de l'itération suivante jusqu'à ce qu'elle [converge](gloss:converge) vers une solution optimale. La nature hybride de cette famille d'algorithmes vient du fait que les fonctions de coût sont évaluées à l'aide de ressources quantiques et optimisées à l'aide de ressources classiques.

1. **Problème d'initialisation** : Les algorithmes variationnels commencent par initialiser l'ordinateur quantique dans un *état par défaut* $|0\rangle$, puis le transforment en un état désiré (non paramétré) $|\rho\rangle$, que nous appellerons *état de référence* .

    Cette transformation est représentée par l'application d'un opérateur de référence unitaire $U_R$ sur l'état par défaut, tel que $U_R|0\rangle = |\rho\rangle$.

2. **Préparer ansatz** : Pour commencer l'optimisation itérative de l'état par défaut $|0\rangle$ à l'état cible $|\psi(\vec\theta)\rangle$, nous devons définir une *forme variationnelle* $U_V(\vec\theta)$ pour représenter une collection d'états paramétrés pour notre algorithme variationnel à explorer.

    Nous nous référons à toute combinaison particulière d'état de référence et de forme variationnelle comme un ansatz, tel que : $U_A(\vec\theta) := U_V(\vec\theta) U_R$. Ansatze prendra finalement la forme de circuits quantiques paramétrés capables de faire passer l'état par défaut $|0\rangle$ à l'état cible $|\psi(\vec\theta)\rangle$.

    En tout nous aurons :

    $$
   \begin{aligned}
   |0\rangle \xrightarrow{U_R} U_R|0\rangle

   & = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle \\[1mm]

   & = U_V(\vec{\theta})U_R|0\rangle \\[1mm]

   & = U_V(\vec{\theta})|\rho\rangle \\[1mm]

   & = |\psi(\vec{\theta})\rangle \\[1mm]

   \end{aligned}
   $$

3. **Évaluer la fonction de coût** : Nous pouvons encoder notre problème dans une *fonction de coût* $C(\vec\theta)$ comme une combinaison linéaire d'opérateurs de Pauli, exécutée sur un système quantique. Bien qu'il puisse s'agir d'informations sur un système physique, comme l'énergie ou le spin, nous pouvons également coder des problèmes non physiques. Nous pouvons tirer parti des primitives Qiskit Runtime pour traiter le bruit avec suppression et atténuation des erreurs tout en évaluant notre fonction de coût.

4. **Optimiser les paramètres** : les évaluations sont transmises à un ordinateur classique, où un optimiseur classique les analyse et choisit le prochain ensemble de valeurs pour les paramètres variationnels. Si nous avons une solution optimale préexistante, nous pouvons la définir comme *point initial* $\vec\theta_0$ pour *amorcer* notre optimisation. L'utilisation de cet *état initial* $|\psi(\vec\theta_0)\rangle$ pourrait aider notre optimiseur à trouver une solution valide plus rapidement.

5. **Ajustez les paramètres d'ansatz avec les résultats, et réexécutez** : l'ensemble du processus est répété jusqu'à ce que les critères de finalisation de l'optimiseur classique soient remplis, et qu'un ensemble optimal de valeurs de paramètres $\vec\theta^ *$ soit renvoyé. L'état solution proposé pour notre problème sera alors $|\psi(\vec\theta^* )\rangle = U_A(\vec\theta^*)|0\rangle$.

## Théorème variationnel

Un objectif commun des algorithmes variationnels est de trouver l'état quantique avec la valeur propre la plus basse ou la plus élevée d'un certain observable. Une idée clé que nous utiliserons est le *théorème variationnel* de la mécanique quantique. Avant d'entrer dans son énoncé complet, explorons une partie de l'intuition mathématique qui le sous-tend.

### Intuition mathématique pour l'énergie et les états fondamentaux

En mécanique quantique, l'énergie se présente sous la forme d'une observable quantique généralement appelée *hamiltonien* , que nous désignerons par $\hat{\mathcal{H}}$. Considérons sa [décomposition spectrale](gloss:decomposition) :

$$
\hat{\mathcal{H}} = \sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|
$$

où $N$ est la dimensionnalité de l'espace des états, $\lambda_{k}$ est la $k$-ième valeur propre ou, physiquement, le $k$-ième niveau d'énergie, et $|\phi_k\rangle$ est l' [état propre](gloss:eigenstate) correspondant : $\hat{\mathcal{H}}|\phi_k\rangle = \lambda_k |\phi_k\rangle$, l'énergie attendue d'un système dans l'état (normalisé) $|\psi\rangle$ sera :

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \langle \psi |\bigg(\sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|\bigg) | \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k \langle \psi |\phi_k\rangle \langle \phi_k| \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

\end{aligned}
$$

Si on prend en compte que $\lambda_0\leq \lambda_k, \forall k$, on a :

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& \geq  \sum_{k=0}^{N-1} \lambda_0 |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \\[1mm]

\end{aligned}
$$

Puisque `{latex} \{ |\phi_k\rangle \}_{k=0}^{N-1}` est une base orthonormée, la probabilité de mesurer $|\phi_{k} \rangle$ est $p_k = |\ langle \psi |\phi_{k} \rangle |^2$, et la somme de toutes les probabilités est telle que $\sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle| ^2 = \sum_{k=0}^{N-1}p_k = 1$. En bref, l'énergie attendue de tout système est supérieure à l'énergie la plus basse ou à l'énergie de l'état fondamental :

$$
\langle \psi | \hat{\mathcal{H}} | \psi \rangle \geq \lambda_0.
$$

L'argument ci-dessus s'applique à tout état quantique valide (normalisé) $|\psi\rangle$, il est donc parfaitement possible de considérer des états paramétrés $|\psi(\vec\theta)\rangle$ qui dépendent d'un vecteur de paramètres $\ vec\theta$. C'est là que la partie "variationnelle" entre en jeu. Si l'on considère une fonction de coût donnée par $C(\vec\theta) := \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle$ et que vous voulez le minimiser, le minimum satisfera toujours :

$$
\min_{\vec\theta} C(\vec\theta) = 
\min_{\vec\theta} \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle \geq \lambda_0.
$$

La valeur minimale de $C(\vec\theta)$ sera la plus proche que l'on puisse obtenir de $\lambda_0$ en utilisant les états paramétrés $|\psi(\vec\theta)\rangle$, et l'égalité ne sera atteinte s'il existe un vecteur de paramètres $\vec\theta^ *$ tel que $|\psi(\vec\theta^* )\rangle = |\phi_0\rangle$

### Théorème variationnel de la mécanique quantique

Si l'état (normalisé) $|\psi\rangle$ d'un système quantique dépend d'un vecteur de paramètre $\vec\theta$, alors l'approximation optimale de l'état fondamental (ie l'état propre $|\phi_0\rangle$ avec le valeur propre minimale $\lambda_0$) est celle qui minimise la [valeur d'espérance](gloss:expectation) du hamiltonien $\hat{\mathcal{H}}$ :

$$
\langle \hat{\mathcal{H}} \rangle(\vec\theta) := 
\langle \psi(\vec\theta) |\hat{\mathcal{H}}| \psi(\vec\theta) \rangle \geq 
\lambda_0
$$

La raison pour laquelle le théorème variationnel est énoncé en termes de minima d'énergie est qu'il comprend un certain nombre d'hypothèses mathématiques :

- Pour des raisons physiques, une borne inférieure finie à l'énergie $E \geq \lambda_0 &gt; -\infty$ doit exister, même pour $N\rightarrow\infty$.
- Les limites supérieures n'existent généralement pas.

Cependant, mathématiquement parlant, il n'y a rien de spécial à propos de l'hamiltonien $\hat{\mathcal{H}}$ au-delà de ces hypothèses, de sorte que le théorème peut être généralisé à d'autres observables quantiques et à leurs états propres à condition qu'ils suivent les mêmes contraintes. Notez également que si des bornes supérieures finies existent, les mêmes arguments mathématiques pourraient être avancés pour maximiser les valeurs propres en remplaçant les bornes inférieures par des bornes supérieures.

Avec cette leçon, vous avez appris la vue de haut niveau des algorithmes variationnels. Au cours des leçons suivantes, nous explorerons chaque étape plus en détail et leurs compromis associés.